In [1]:
import os
import pandas as pd

path_dir = "label/UCF-101"

def new_csv(path_dir):
    '''
    [Args]
    path_dir : path to folder contain train.csv, val.csv and newClassInd.txt which hold labels of class remained

    [Result]
    Create new_train.csv and new_val.csv save at path_dir 

    '''
    # Read path
    train = os.path.join(path_dir,'train.csv')
    val = os.path.join(path_dir,'val.csv')
    new_class = os.path.join(path_dir,'newClassInd.txt')
    new_train = os.path.join(path_dir,'new_train.csv')
    new_val = os.path.join(path_dir,'new_val.csv')

    if not os.path.exists(new_train):
        open(new_train, 'w').close() 
    if not os.path.exists(new_val):
        open(new_val, 'w').close() 

    df_csv_train= pd.read_csv(train)
    df_csv_val = pd.read_csv(val)

    # Take lables to keep in new file csv
    with open(new_class,'r') as f:
        labels_keep = set(line.strip().split(' ')[1] for line in f)

    # Filter train.csv
    df_filtered_train = df_csv_train[df_csv_train['label'].isin(labels_keep)]
    df_filtered_train.to_csv(new_train, index=False)

    # Filter val.csv
    df_filtered_val = df_csv_val[df_csv_val['label'].isin(labels_keep)]
    df_filtered_val.to_csv(new_val, index=False)

new_csv(path_dir)




In [1]:
import glob


file = glob.glob('data/UCF-101-DB/*/*/n_frames')

In [4]:
temp = []

for f in file:
    with open(f) as p:
        temp.append(int(p.read()))

In [7]:
import numpy as np 

np.mean(temp)

186.6584084084084

In [1]:
import math
import random

class LoopPadding(object):

    def __init__(self, size):
        self.size = size

    def __call__(self, frame_indices):
        out = frame_indices

        for index in out:
            if len(out) >= self.size:
                break
            out.append(index)

        return out

class TemporalEvenCrop(object):

    def __init__(self, size, n_samples=1, percentage = .5):
        self.size = size
        self.n_samples = n_samples
        self.loop = LoopPadding(size)
        self.per = percentage

    def sampling(self, frame_count):

        frame_indices = [i for i in range(frame_count)]

        n_frames = int(len(frame_indices)*self.per)
        stride = max(
            1, math.ceil((n_frames - 1 - self.size) / (self.n_samples - 1)))
        
        # stride = 1

        out = []
        for begin_index in frame_indices[::stride]:
            if len(out) >= self.n_samples:
                break
            end_index = min(frame_indices[-1] + 1, begin_index + self.size)
            sample = list(range(begin_index, end_index))

            if len(sample) < self.size:
                out.append(self.loop(sample))
                break
            else:
                out.append(sample)

        return out

In [1]:
import os
import glob

def num_images(dataset_path,dst_path):

    if not os.path.exists(dst_path):
        os.makedirs(dst_path)

    for image_file in glob.glob(dataset_path+'/*/*/'):
        frames_count =0
        for files in os.listdir(image_file):
            if files.endswith('.jpg'):
                frames_count +=1

        if frames_count<=0:
            print('{} does not have any frames'.format(image_file))
            continue
        
        count_frame_path = os.path.join(dst_path,image_file.split('/',maxsplit=1)[-1])
        if not os.path.exists(count_frame_path):
            os.makedirs(count_frame_path)

        with open(os.path.join(count_frame_path,'n_frames'),'w') as dst_file:
            dst_file.write(str(frames_count))

num_images("UCF-101-JPG","UCF-101-JPG")

In [2]:
class SequenceSampler(object):
    def __init__(self, len_scale, frame_skip = 4, **kwags):

        assert  len_scale < frame_skip, "len_scale should be smaller than frame_skip"

        self.len_scale = len_scale
        self.frame_skip = frame_skip
        self.lenght_video = 187

    def sampling(self, video_len):
        
        start_index = 0

        end_index = int(self.lenght_video)

        offset = 0
        if end_index > video_len:
            offset = end_index - video_len

            indices = [i for i in range(start_index, video_len)]
        
            indices += [video_len - 1 for i in range(0, offset)]
        else:
            indices = [i for i in range(start_index, end_index)]

        # indices = [indices[i] for i in range(0, len(indices), self.frame_skip + 1)]
        # return indices
        indices_list = []
        for i in range(0, len(indices), self.frame_skip + self.len_scale):
            indices = []
            for j in range(self.len_scale):
                indices.append(i + j)
            indices_list.append(indices)

        return indices_list

In [6]:
a = SequenceSampler(2, 4)
a.sampling(187)

[[0, 1],
 [6, 7],
 [12, 13],
 [18, 19],
 [24, 25],
 [30, 31],
 [36, 37],
 [42, 43],
 [48, 49],
 [54, 55],
 [60, 61],
 [66, 67],
 [72, 73],
 [78, 79],
 [84, 85],
 [90, 91],
 [96, 97],
 [102, 103],
 [108, 109],
 [114, 115],
 [120, 121],
 [126, 127],
 [132, 133],
 [138, 139],
 [144, 145],
 [150, 151],
 [156, 157],
 [162, 163],
 [168, 169],
 [174, 175],
 [180, 181],
 [186, 187]]